# Init

In [2]:
import datatable as dt
import multiprocessing as mp
import numpy as np
import spacy
import time

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from datatable import f
from functools import partial
from spacy.tokens import Doc, DocBin, Span
from tqdm.auto import tqdm

dt.init_styles()

WORK_DIR = '/home/yu/OneDrive/CC'
DATA_DIR = f'{WORK_DIR}/data'
os.chdir(WORK_DIR)

# Convert Doc to "text tokens"

In [ ]:
nlp = spacy.load('en_core_web_lg')

doc = nlp('She likes movies.')

doc.has_annotation("DEP")

In [5]:
# Load DocBin from disk
nlp = spacy.load('en_core_web_trf')
# docs = list(DocBin(store_user_data=True).from_disk('data/doc_sp500_test.spacy').get_docs(nlp.vocab))

docs = []
for _ in range(1):
    docs.extend(list(DocBin(store_user_data=True).from_disk(f'data/doc_sp500_trf_{_}.spacy').get_docs(nlp.vocab)))

# register extension for Doc
Doc.set_extension('transcriptid', default=None, force=True)

# Register extension for Span
Span.set_extension('transcriptid', default=None, force=True)
Span.set_extension('componentid', default=None, force=True)
Span.set_extension('componentorder', default=None, force=True)
Span.set_extension('componenttypeid', default=None, force=True)
Span.set_extension('speakerid', default=None, force=True)
Span.set_extension('speakertypeid', default=None, force=True)
Span.set_extension('is_component', default=False, force=True)

In [3]:
# Select componentid that belongs to MD and QA
ld('text_component_sp500', ldname='text_component')
text_component = dt.Frame(text_component)

# componentid: Management Discussion
componentids_md = text_component[(f.transcriptcomponenttypeid==2) & (f.speakertypeid==2), f.transcriptcomponentid].to_list()[0]

# componentid: Q & A
componentids_qa = text_component[((f.transcriptcomponenttypeid==3) | (f.transcriptcomponenttypeid==4)) & ((f.speakertypeid==2)|(f.speakertypeid==3)), f.transcriptcomponentid].to_list()[0]

"text_component_sp500.feather" (978.0 MB) loaded as "text_component" (2s)


In [21]:
# Convert Doc to "text tokens"

# Filtering Rule:
# - only keep lemma
# - no stop words (stop words is informative while comparing)
# - no punctuation
# - no "like number"

def make_text_tokens(docs, componentids):
    texttoken = {}
    
    # For every doc, join the required spans into a list of str
    for doc in tqdm(docs):
        txttok = []
        for span in doc.spans['components']:
            if span._.componentid in componentids:
                txttok.extend([t.lower_ for t in span 
                if ((not t.is_punct) & (not t.like_num) & (not t.is_stop))])

        # If no text found, add an empty str
        if len(txttok)==0:
            txttok = ['']

        # return
        texttoken[doc._.transcriptid] = txttok
    
    return texttoken

texttoken_md = make_text_tokens(docs[:1000], componentids_md)
# texttoken_qa = make_text_tokens(docs, componentids_qa)

100%|██████████| 1000/1000 [01:25<00:00, 11.71it/s]


In [24]:
# Convert Doc to "text tokens"

# Filtering Rule:
# - only keep lemma
# - no stop words (stop words is informative while comparing)
# - no punctuation
# - no "like number"

def make_text_tokens(doc, componentids):
    
    # For every doc, join the required spans into a list of str
    txttok = []
    for span in doc.spans['components']:
        if span._.componentid in componentids:
            txttok.extend([t.lower_ for t in span 
            if ((not t.is_punct) & (not t.like_num) & (not t.is_stop))])

    # If no text found, add an empty str
    if len(txttok)==0:
        txttok = ['']

    # return 
    return {doc._.transcriptid: txttok}

with ProcessPoolExecutor(max_workers=16) as executor:
    texttoken_md = list(tqdm(executor.map(partial(make_text_tokens, componentids=componentids_md), docs[:1000], chunksize=100), total=1000))


100%|██████████| 1000/1000 [01:00<00:00, 16.62it/s]


# Convert to DTM

In [8]:
'''
from sklearn.feature_extraction.text import CountVectorizer

# Convert to DTM

# Setting:
# - keep ALL tokens
vectorizer = CountVectorizer(preprocessor=lambda x: x,
                             tokenizer=lambda x: x,
                             lowercase=False,
                             ngram_range=(1,2))

# Learn vocabulary 
vectorizer.fit(texttoken_md.values())
vectorizer.fit(texttoken_qa.values())

# Make DTM
dtm_md = vectorizer.transform(texttoken_md.values())
dtm_qa = vectorizer.transform(texttoken_qa.values())

# Compute similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity = np.diag(cosine_similarity(dtm_md, dtm_qa))
similarity

'''

'\nfrom sklearn.feature_extraction.text import CountVectorizer\n\n# Convert to DTM\n\n# Setting:\n# - keep ALL tokens\nvectorizer = CountVectorizer(preprocessor=lambda x: x,\n                             tokenizer=lambda x: x,\n                             lowercase=False,\n                             ngram_range=(1,2))\n\n# Learn vocabulary \nvectorizer.fit(texttoken_md.values())\nvectorizer.fit(texttoken_qa.values())\n\n# Make DTM\ndtm_md = vectorizer.transform(texttoken_md.values())\ndtm_qa = vectorizer.transform(texttoken_qa.values())\n\n# Compute similarity\nfrom sklearn.metrics.pairwise import cosine_similarity\nsimilarity = np.diag(cosine_similarity(dtm_md, dtm_qa))\nsimilarity\n\n'